# CNN for leave-one-out training splits, time and circadian estimates included
We generated the PSG, time, and spectrogram data pickles using [this ETL/feature extraction script](https://github.com/ericcanton/sleep_classifiers_accel_ML/blob/master/source/etl.py). Our data loading, training, and evaluation scheme is described below. If you want to dive into using the code, skip to "Preamble". 

To get the data, you can add following folder to your Google Drive from Eric Canton's Google Drive. 
*    https://drive.google.com/drive/folders/1pYoXsFHhK1X3qXIOCiTrnoWKG6MTNucy?usp=sharing  

This should have two folders: pickles and neural. 
1. For now, neural is empty. Trained neural networks appear here with naming format like "123_trained_cnn.h5" for subject number 123. 
2. pickles contains one folder per subject. We recommend leaving out 7749105 because only about an hour of data is recorded for them (watch failure) by setting <code>exclude = ['7749105']</code> to <code>data_yielder</code> in the Training section's for loops and the function calls in the Evaluation section; this is the case at time of sharing. 



## 0. Processed Data loading, model training, and evaluation scheme

### 0.1 Evaluation and split generation functions
Basically, these "data pickles" are saved numpy arrays containing our data points stored along axis 0. To easily generate leave-one-out train/test splits, we have divided the data into separate folders for each subject in the study. in <code>evaluators.py</code> we have defined two generators, 

*   <code>data_yielder</code> whose <code>next()</code> method returns a tuple
    that depends on the <code>nn</code> parameter of this Yielder. 
    If <code>nn=True</code> (the default) when called, the tuple is

 <code>(spectrogram[i], time[i], psg labels[i], neural network[i]) </code>  
    enumerated according with the order returned by <code>os.walk</code> on 
    the <code>data_path</code> folder. The first three elements are numpy
    arrays of shape (\*, 65, 73), (\*, 1), (\*, 1); the last is a keras.model
    trained on the data for all subjects except subject[i]. 

    If <code>nn=False</code> then returns just the first three elements above,

 <code>(spectrogram[i], time[i], psg labels[i]).</code>  

    This default mode is intended to provide testing splits and neural networks 
    for evaluation. Specifically, the function <code>pr_roc_from_path</code>
    was written with a <code>data_yielder</code> instance to be passed as 
    the first argument, though it functions perfectly well for any list-like
    Python object (generally, anything having a <code>next()</code> method will
    function as well). See the next subsection for an explanation of this
    function. 

*   <code>split_yielder</code> calls <code>data_yielder</code>, collects all
    the elements from <code>data_yielder</code>, then yields tuples 

    <code>(subject number excluded, training_split_data, testing_split_data)</code>

    where the data from subject number <code>subject number excluded</code> 
    is <code>testing_split_data</code> and everyone else's data is stacked
    into <code>training_split_data</code>. If <code>t</code> is one of these
    splits, then it is a tuple with elements:
    
    *   <code>t[0]</code> is the model input data, the structure 
    depending on the <code>with_time</code> parameter. If 
    <code>with_time=True</code> (the default) 
    this is a tuple of length 2: <code>t[0][0]</code> are the spectrograms
    as a numpy array with shape (*, 65, 73) and <code>t[0][1]</code> are the
    timestamps, as a numpy array with shape (*,1) or possibly (*,). If 
    <code>with_time=False</code> then <code>t[0]</code> is just the 
    array of spectrograms.
    
    *   <code>t[1]</code> is the numpy array of PSG labels, having values in
    <code>[0, 1, 2, 3, 4, 5]</code> with 0 indicating awake and 5 indicating
    REM. Thus, you likely need to process <code>t[1]</code> more using 
    <code>np.where</code> or <code>np.piecewise</code> to labels you 
    want. For sleep-wake labeling, try  
        <code>psg_sleep_wake = np.where(t[1] > 0, 1, 0)</code>  

The data pickles being yielded above are stored in the <code>pickles</code> folder inside the Drive folder pointed to by <code>data_path</code>, defined below. The data for subject <code>123</code> is stored in the folder <code>data_path/pickles/123</code>; there you will find three files:


1.   <code>psg.pickle</code>
2.   <code>spectros.pickle</code>
3.   <code>time.pickle</code>




### 0.2 Explanation of the pickles

Assume we have loaded these three pickles for subject <code>123</code> into the variables <code>psg, spectros,</code> and <code>time</code>. These are all numpy arrays whose axis-0 lengths are the same.  
*   <code>psg[i]</code> is the PSG label for the time window <code>[30\*i, 30\*(i+1)]</code>. 
*   <code>time[i]</code> is equal to <code>30\*i + 15</code>, the center of this time window, and 
*    <code>spectros'[i]</code> is the spectrogram of derivative of magnitude of 
    acceleration (quite a mouth full...) for times ranging between 
    <code>30\*i + 15 - 46.08</code> and <code>30\*i + 15 + 46.08</code>. 
    The somewhat strange number <code>46.08 = 92.16/2</code> is a consequence of wanting 90 second windows with data sampling rate of 50Hz, but also wanting the number of records in this window (a priori, 90/0.02 = 4500) to be divisble by 128, the number of points per Fourier transform used in the spectrogram creation. So, the shortest time interval over 90 seconds satisfying the additional divisibility requirement is 92.16 seconds, since 4500 = 20 mod 128, or 4608 = 4500 + (128 - 20). 

### 0.3 Olivia: Adding circadian predictions
*Olivia: you should save the circadian values for each window in this folder, too. Probably easiest to modify <code>etl.py</code> adding a step to load and then save the circadian value in a separate pickle. Or, you could add a column to each time.pickle, giving that array shape (?, 2) instead of (?, 1). I'll indicate in the cell below defining our CNN how to modify things in either case.*

## 1. Preamble

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve as pr_curve

import tensorflow as tf
from tensorflow.keras.layers import concatenate, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D 
from tensorflow.keras.models import Model

import pickle
import glob
import sys

In [ ]:
# See if a GPU is available. 
# This speeds up training the NNs, with significant gains on the CNN.
gpu = tf.test.gpu_device_name()
if "GPU" in gpu:
    print(gpu)
else:
    print("No GPU found!")

### 1.1 Define paths for finding data and support libraries
These paths should be from a mounted Google Drive. You should change <code>data_path</code> to point to the directory where your copies of the pickled spectrograms/PSG labels are stored, and 

In [ ]:
data_path = "/content/drive/My Drive/sleep_classifiers_accel_ML/data/"
source_path = "/content/drive/My Drive/sleep_classifiers_accel_ML/source/"

In [ ]:
sys.path.append(source_path)

The following library should be stored as 
<code>source_path + "evaluator.py"<c/code> and is obtainable from <a href="https://github.com/ericcanton/sleep_classifiers_accel_ML/blob/master/source/evaluator.py">Eric's GitHub</a>.

In [ ]:
import evaluator as ev

## 2. CNN Definitions
We define two CNNs, one for sleep/wake classification, the second for staging. These both incorporate time, and can easily be modified to incorporate further inputs. I have detailed how one would do this in model 2.1 immediately below.

### 2.1 CNN for predictiong sleep/wake using time
The last layer has 2 neurons because we want to predict 2 classes: 
*   0 for "awake"
*   1 for "asleep"

In [ ]:
##
# Model input layers
# The shape parameters do not include the number of training samples, 
# which should be the length of axis-0 of the input numpy tensors. 
##

# Spectrograms. These must be reshaped using .reshape(-1, 65, 73, 1)
# from their original shape (-1, 65, 73) to make them appear to be "black and white images"
# to the Conv2D layers below. 
inputs_spectro = Input(shape=(65, 73, 1)) 
inputs_time = Input(shape=(1,))
#####
#####
# Add more layers here. For example, if we wanted to add
# the original x,y,z accleration data (averaged over the time window, e.g.)
# then we would add
#   inputs_accel = Input(shape(3,))
#####
#####

# Convolutional layers. Operate only on spectrograms
c_1 = Conv2D(64, kernel_size=3, activation='relu', data_format='channels_last')(inputs_spectro)
m_1 = MaxPool2D()(c_1)
c_2 = Conv2D(32, kernel_size=3, activation='relu', data_format='channels_last')(m_1)
m_2 = MaxPool2D()(c_2)

# The output of m_2 is a stream of two-dimensional features
# Densely connected layers want one-dimensional features, so we "unwind"
# the two-dimensional features using a Flatten layer:
#   ***
#   ***
#   ***
#   *** ~~~> ***,***,***,***
flat = Flatten(data_format='channels_last')(m_2)

# Now append time to the front of this flattened data
concat = concatenate([ \
#####
#####
# If you added layers above, include them in the list inside concatenate.
# Order shouldn't really matter, just add them below. 
#####
#####
                      inputs_time, \
                      flat])
    
# Start of densely connected layers
d_1 = Dense(32, activation='relu')(concat)
drop = Dropout(0.05)(d_1)
d_2 = Dense(32, activation='relu')(drop)
d_3 = Dense(32, activation='relu')(d_2)

# Two categories predicted, giving 2 neurons. activation='softmax' makes
# these two nodes sum to 1 and be bounded between 0 and 1.
out = Dense(2, activation='softmax')(d_3)

cnn_time = Model(inputs=[inputs_spectro, inputs_time], outputs=out)
cnn_time.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
# Save this trained model to the attached Google drive. Since the weights in
# each layer of the model are randomly initialized, this is important to ensure
# we're starting with exactly the same pre-training state to compare performance
# of *this exact model instance* on the training splits.
cnn_time.save(source_path + "cnn_model_binary_softmax.h5")

### 2.2 CNN for W/N12/N34/REM staging
The model below is identical to the one in 2.1 aside from there being 4 neurons
on the last Dense layer corresponding to the four stages of sleep above. These 
stages MUST be <code>[0,1,2,3]</code> in the PSG labeling for the training.

In [ ]:
# Model input layers
inputs_spectro = Input(shape=(65, 73, 1))
inputs_time = Input(shape=(1,))

# Convolutional layers. Operate only on spectrograms
c_1 = Conv2D(64, kernel_size=3, activation='relu', data_format='channels_last')(inputs_spectro)
m_1 = MaxPool2D()(c_1)
c_2 = Conv2D(32, kernel_size=3, activation='relu', data_format='channels_last')(m_1)
m_2 = MaxPool2D()(c_2)

# Flatten the layer to prepare for the densely connected layers
flat = Flatten(data_format='channels_last')(m_2)
# Now append time to the front of this flattened data
concat = concatenate([inputs_time, flat])
    
# Start of densely connected layers
d_1 = Dense(32, activation='relu')(concat)
drop = Dropout(0.05)(d_1)
d_2 = Dense(32, activation='relu')(drop)
d_3 = Dense(32, activation='relu')(d_2)

# Two categories predicted
out = Dense(4, activation='softmax')(d_3)

cnn_time = Model(inputs=[inputs_spectro, inputs_time], outputs=out)
cnn_time.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
cnn_time.save(source_path + "cnn_model_staging_softmax.h5")

### 1.2 No time
Same as the first, but without time and so no concatenate layer between Flatten and Dense. 

In [ ]:
# Model input layers
inputs_spectro = Input(shape=(65, 73, 1))

# Convolutional layers. Operate only on spectrograms
c_1 = Conv2D(64, kernel_size=3, activation='relu', data_format='channels_last')(inputs_spectro)
m_1 = MaxPool2D()(c_1)
c_2 = Conv2D(32, kernel_size=3, activation='relu', data_format='channels_last')(m_1)
m_2 = MaxPool2D()(c_2)

# Flatten the layer to prepare for the densely connected layers
flat = Flatten(data_format='channels_last')(m_2)
    
# Start of densely connected layers
d_1 = Dense(32, activation='relu')(flat)
drop = Dropout(0.05)(d_1)
d_2 = Dense(32, activation='relu')(drop)
d_3 = Dense(32, activation='relu')(d_2)

# Two categories predicted
out = Dense(2, activation='softmax')(d_3)

cnn_no_time = Model(inputs=inputs_spectro, outputs=out)
cnn_no_time.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
cnn_no_time.save(source_path + "cnn_no_time_model_softmax.h5")

## 2. Densely connected network definition
To make somewhat comparable capacity between models we replace the Conv2D
layers in the above models with Dense layers having the same number of neurons.
Basically, the expressive power of the network; too much capacity gives better training results but overfits. There is probably more hyperparameter work to be
done here to assess the correct capacity for the DNN; how much is needed to 
have similar performance as CNNs? Mathematically, this is possible, but 
the width and depth may be exceedingly large. 

See <a href="http://www2.math.technion.ac.il/~pinkus/papers/neural.pdf">Leshno, et al. (1993)</a>  
 *Multilayer Feedforward Networks with a Nonpolynomial Activation Function can Approximate Any Function* (Neural Networks, Vol 6, pp 861--867)


In [ ]:
# Input layers
inputs_spectro = Input(shape=(65, 73))
inputs_time = Input(shape=(1,))

# Flatten the spectrogram input
flat = Flatten()(inputs_spectro)

# Pass spectrograms through several densely connected layers before incorporating time.
d_1 = Dense(64, activation='relu')(flat)
#drop_1 = Dropout(0.2)(d_1)
d_2 = Dense(32, activation='relu')(d_1)
#m_2 = MaxPool1D()(d_2)

# Now append time to the front of this flattened data
concat = concatenate([inputs_time, d_2])
    
# Start of densely connected layers
d_3 = Dense(32, activation='relu')(concat)
drop = Dropout(0.05)(d_3)
d_4 = Dense(32, activation='relu')(drop)
d_5 = Dense(32, activation='relu')(d_4)

# Two categories predicted
out = Dense(2, activation='softmax')(d_3)

dnn_time = Model(inputs=[inputs_spectro, inputs_time], outputs=out)
dnn_time.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
dnn_time.save(source_path + "dnn_with_time.h5")

## 3. Train networks on leave-one-out splits
The loops below use <code>ev.split_yielder</code> to train the above-defined CNNs and DNNs on the training data, per the description of this function in section 0.1. 

In [ ]:
# These describe the main part of the NN naming scheme for saving the trained neural networks.
# For example, the CNN for sleep/wake classification would be trained on the training split excluding
# subject 123, then saved to data_path + "pickles/123/123_softmax_time_cnn.h5"
cnn_time_base = "_softmax_time_cnn.h5"
cnn_no_time_base = "_softmax_time_no_cnn.h5"

dnn_time_base = "_softmax_time_dnn.h5"

In [ ]:
count = 0
for split in ev.split_yielder(data_path=data_path, exclude = ['7749105']):
    # split == (subject in testing_split_data, training_split_data, testing_split_data)
    # split[1] == 2-tuple with element [i]...
    #   [0] : [training spectrograms, training time]
    #   [1] : training psg
    # Same elements for split[2], the testing data
    train_spectros = split[1][0][0].reshape(-1, 65, 73, 1)
    train_time = split[1][0][1].reshape(-1,1)
    train_X = [train_spectros, train_time]
    train_psg = np.where(split[1][1] > 0, 1, 0)

    test_spectros = split[2][0][0].reshape(-1, 65, 73, 1)
    test_time = split[2][0][1].reshape(-1,1)
    test_X = [test_spectros, test_time]
    test_psg = np.where(split[2][1] > 0, 1, 0)

    count += 1 # Keeps track of progress through the training regime
    print("Working on %s (%d of 28)" % (split[0], count))

    split_cnn_time = tf.keras.models.load_model(source_path + "cnn_model_softmax.h5")
    split_cnn_no_time = tf.keras.models.load_model(source_path + "cnn_no_time_model_softmax.h5")
    print("Neural networks loaded.")

    with tf.device(gpu):
        split_cnn_time.fit(train_X, train_psg, epochs = 15, verbose = 1, validation_data = (test_X, test_psg))
        split_cnn_no_time.fit(train_spectros, train_psg, epochs = 15, verbose = 1, validation_data = (test_spectros, test_psg))
    split_cnn_time.save(data_path + "neural/" + split[0] + cnn_time_base)
    split_cnn_no_time.save(data_path + "neural/" + split[0] + cnn_no_time_base)

In [ ]:
count = 0
for split in ev.split_yielder(data_path=data_path, exclude = ['7749105']):
    # split == (subject in testing_split_data, training_split_data, testing_split_data)
    # split[1] == 2-tuple with element [i]...
    #   [0] : [training spectrograms, training time]
    #   [1] : training psg
    # Same elements for split[2], the testing data
    train_spectros = split[1][0][0]
    train_time = split[1][0][1].reshape(-1,1)
    train_X = [train_spectros, train_time]
    train_psg = np.where(split[1][1] > 0, 1, 0)

    test_spectros = split[2][0][0]
    test_time = split[2][0][1].reshape(-1,1)
    test_X = [test_spectros, test_time]
    test_psg = np.where(split[2][1] > 0, 1, 0)

    count += 1 # Keeps track of progress through the training regime
    print("Working on %s (%d of 28)" % (split[0], count))

    split_dnn_time = tf.keras.models.load_model(source_path + "dnn_with_time.h5")
    print("Neural networks loaded.")

    split_dnn_time.fit(train_X, train_psg, epochs = 15, verbose = 1, validation_data = (test_X, test_psg))
    split_dnn_time.save(data_path + "neural/" + split[0] + dnn_time_base)

## 4. Evaluate
We use the <code>pr_roc_from_path</code> function defined in <code>evaluator.py</code> to loop over the trained neural networks with the left-out subject's data as testing data. Passing <code>saveto=data_path + "abc.png"</code> will write the generated PR or ROC curve as abc.png, saved in the folder pointed to by <code>data_path</code>. If <code>saveto=None</code> (default) no picture is saved, just displayed below the cell.

Options for <code>mode</code> are <code>"roc"</code> (the default) and <code>"pr"</code> that determine the obvious evaluation plot type. The output ROC or PR curve has many orangish-yellow curves and one blue curve. The blue curve is the pointwise mean of the other curves. 

In [ ]:
ev.pr_roc_from_path(
    ev.data_yielder(data_path, exclude = ['7749105'], nn_base_name =cnn_time_base), \
    title="Leave-one-out training splits using time: ROC curve for CNN", \
    # Change this to whatever the class label is you want considered "positive" (default: 0 = sleep) \
    pos_label=0, \
    # used for y/x axis labeling. If None, defaults to "True/False positive rate"
    label_names = {'positive' : "Awake", 'negative' : "Sleep"}, \
    # Do not save the picture. Change this a string with full path of image file you want to write
    saveto=None, \
    # make ROC plot instead of PR plot with ="pr"
    mode="roc", \
    # we build our NNs with softmax activation at the final layer
    from_logits=False) 
    